<a href="https://colab.research.google.com/github/denidiana2020/Machine_Learning/blob/main/Klasifikasi_kegagalan_sistem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Klasifikasi kegagalan sistem


Sebagai pelatihan tema praktis yang baik menggunakan pembelajaran mesin, Mengidentifikasi dan memprediksi penyebab kegagalan sistem tekanan udara.

Notebook ini melakukan kegagalan sistem klasifikasi oleh LGBM
EDA dan pra-pemrosesan data dikonfirmasi oleh kepentingan variabel dengan hutan acak dan pengelompokan dan PCA. Akibatnya, terapkan metode yang tampaknya baik untuk data latih ke data uji dan konfirmasi.
Selain itu, untuk ketidakseimbangan data, konfirmasi efeknya dengan metode seperti oversampling dan undersampling.

# Import Library

In [ ]:
#library Sederhana
import numpy as np
import pandas as pd
import time
import warnings
warnings.simplefilter("ignore")

# Visualization
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
import seaborn as sns

# statiscics
import scipy

# Data preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Grid search
from sklearn.model_selection import GridSearchCV

# StratifiedKFold
from sklearn.model_selection import StratifiedKFold

# Learning curve
from sklearn.model_selection import learning_curve

# Validation curve
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from scipy import interp

# Dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Classification method
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

# Imbalanced data preprocessing
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

# Validation
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Dataloading dan cek Basis data¶

content
Set pelatihan berisi total 60000 contoh di mana 59000 termasuk dalam kelas negatif dan 1000 kelas positif. Set tes berisi 16000 contoh. Ada 171 atribut per record.

Nama atribut data telah dianonimkan karena alasan kepemilikan. Ini terdiri dari penghitung numerik tunggal dan histogram yang terdiri dari tempat sampah dengan kondisi berbeda. Biasanya histogram memiliki kondisi ujung terbuka di setiap ujungnya. Misalnya, jika kita mengukur suhu lingkungan "T" maka histogram dapat didefinisikan dengan 4 bin di mana:

Atributnya adalah sebagai berikut: kelas, kemudian data operasional yang dianonimkan. Data operasional memiliki pengenal dan id bin, seperti "Identifier_Bin". Total ada 171 atribut, dimana 7 adalah variabel histogram. Nilai yang hilang dilambangkan dengan "na".

Ucapan Terima Kasih
File ini adalah bagian dari Kegagalan APS dan Data Operasional untuk Truk Scania. Itu diimpor dari Repositori UCI ML.

Inspirasi
Total biaya model prediksi jumlah Biaya_1 dikalikan dengan jumlah Instance dengan kegagalan tipe 1 dan Biaya_2 dengan jumlah instans dengan kegagalan tipe 2, menghasilkan Total_cost. Dalam hal ini Biaya_1 mengacu pada biaya pemeriksaan yang tidak perlu yang perlu dilakukan oleh mekanik di bengkel, sedangkan Biaya_2 mengacu pada biaya kehilangan truk yang rusak, yang dapat menyebabkan kerusakan. Biaya_1 = 10 dan Biaya_2 = 500, dan Total_cost = Cost_1No_Instances + Cost_2No_Instances.

In [ ]:
#upload file dataset training dan test
%cd sample_data/
from google.colab import files

uploaded = files.upload()

[Errno 2] No such file or directory: 'sample_data/'
/content/sample_data


In [ ]:
!ls

In [ ]:
# train data
train = pd.read_csv("/content/sample_data/aps_failure_training_set.csv")
# test data
test = pd.read_csv("/content/sample_data/aps_failure_test_set.csv")

In [ ]:
# datacheck
train.head()

In [ ]:
# data size
train.shape

In [ ]:
# null value
train.isnull().sum().sum()

In [ ]:
# data info
train.dtypes

#Data prepcocessing

Untuk sementara, 'na' berubah menjadi nilai Null.
Setelah memahami seluruh data, pertimbangkan pemrosesan nilai nol.

In [ ]:
# train data
train = pd.read_csv("/content/sample_data/aps_failure_training_set.csv", na_values="na")
# test data
test = pd.read_csv("/content/sample_data/aps_failure_test_set.csv",na_values="na")

In [ ]:
# null check
col = train.iloc[:,1:].columns
null_ratio = train.iloc[:,1:].isnull().sum().values / train.shape[0]*100

In [ ]:
# Check by visualization
plt.figure(figsize=(20,8))
plt.plot(col, null_ratio)
plt.xlabel("variables")
plt.ylabel("ratio(%)")
plt.xticks(rotation=90, fontsize=10)
plt.title("Null ratio")
plt.legend()

Beberapa variabel memiliki persentase nilai Null yang sangat tinggi. Bagaimana cara menggunakan informasi ini? Perlu pertimbangan.

In [ ]:
# data type check
train.dtypes

# EDA
Pemeriksaan jumlah kelas

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(train["class"])
plt.title("neg:{0} / pos:{1}".format(train["class"].value_counts()[0], train["class"].value_counts()[1]))

Periksa distribusi variabel secara numerik
Jumlah variabel adalah 171 yang sangat besar, jadi periksa dulu distribusi variabel secara numerik untuk mendapatkan karakteristiknya.

Ganti label kelas dengan angka untuk analisis nanti. 1: positif, 0: negatif.

In [ ]:
# define function
def class_flg(x):
    if x["class"] == 'pos':
        res = 1
    else:
        res = 0
    return res

train["class"] = train.apply(class_flg, axis=1)
# test data
test["class"] = test.apply(class_flg, axis=1)

Memahami distribusi dan hubungan setiap data¶

In [ ]:
# min and max and mean and std
col = train.iloc[:,1:].columns
mean = train.iloc[:,1:].mean()
min_ = train.iloc[:,1:].min()
max_ = train.iloc[:,1:].max()
std = train.iloc[:,1:].std()

In [ ]:
# Visualization by plot
plt.figure(figsize=(25,8))

plt.plot(col, mean, linewidth=5, color="blue", label='mean') #mean
plt.fill_between(col, mean+std, mean-std, alpha=0.15, color='blue', label='±1σ') # ±1σ
plt.plot(col, min_, linewidth=5, color='red', linestyle='--', label='max-min') # min
plt.plot(col, max_, linewidth=5, color='green', linestyle='--') # max
plt.xlabel("variables")
plt.ylabel("values")
plt.yscale("log")
plt.xticks(rotation=90, fontsize=10)
plt.title("variables range")
plt.legend()

Ini bukan grafik yang terorganisir, tetapi Anda dapat melihat ukuran dan lebar variabel. Nilai absolut dari setiap nilai dapat berkisar dari angka yang cukup besar hingga yang kecil. Bahkan jika Anda menggunakan grafik logaritmik, grafiknya telah berubah secara signifikan, jadi berhati-hatilah saat menangani setiap skala.

**Perbedaan menurut label**

In [ ]:
train_mean = train.groupby("class").mean().T
train_std = train.groupby("class").std().T

# Visualization by plot
plt.figure(figsize=(25,8))

plt.plot(train_mean.index, train_mean[1], linewidth=5, color="red", label='pos') #mean
plt.fill_between(train_mean.index, train_mean[1]+train_std[1], train_mean[1]-train_std[1], alpha=0.15, color='orange', label='±1σ') # ±1σ

plt.plot(train_mean.index, train_mean[0], linewidth=5, color="blue", label='nag') #mean
plt.fill_between(train_mean.index, train_mean[0]+train_std[0], train_mean[0]-train_std[0], alpha=0.15, color='green', label='±1σ') # ±1σ

plt.xlabel("variables")
plt.ylabel("values")
plt.yscale("log")
plt.xticks(rotation=90, fontsize=10)
plt.title("variables range")
plt.legend()

# Correlation
Next, check the correlation of each variable with a heat map

In [ ]:
# Null data are tempolary filled mean value.
matrix = train.iloc[:,1:].iloc[1:].corr()
plt.figure(figsize=(15,15))
sns.heatmap(matrix, vmax=1, vmin=-1, cmap='bwr', square=True, annot=False, center=0, yticklabels=False, xticklabels=False)

Terlihat dari banyaknya variabel yang memiliki korelasi yang tinggi. Jika isi variabel dan hubungan teknisnya jelas, variabel dapat dipilih, tetapi kali ini tidak ada informasi sebelumnya. Oleh karena itu, perlu dilakukan ekstraksi variabel yang relevan dengan menggunakan analisis data.

# Skerness and kurtosis


In [ ]:
# skew
col = train.iloc[:,1:].columns
# Roop, calculate with drop na values.
skew = []
for i in col:
    sk = scipy.stats.skew(train[i].dropna())
    skew.append(sk)

In [ ]:
# kurtosis
# Roop, calculate with drop na values.
kurt = []
for i in col:
    ku = scipy.stats.kurtosis(train[i].dropna())
    kurt.append(ku)

In [ ]:
# check with graph
fig, ax = plt.subplots(1, 2, figsize=(20,6))
sns.distplot(skew, ax=ax[0], kde=False, bins=100)
ax[0].set_xlabel("Skewness")
ax[0].set_ylabel("Frequency")
ax[0].set_title("Skewness")
sns.distplot(kurt, ax=ax[1], kde=False, bins=100)
ax[1].set_xlabel("Kurtosis")
ax[1].set_ylabel("Frequency")
ax[1].set_title("Kurtosis")

Distribusi data dikonfirmasi. Pertama, hanya ditemukan nilai skewness yang lebih besar dari 0, dan sebarannya memiliki ekor kanan yang panjang dan bias ke kiri. Dikonfirmasi juga bahwa semua nilai sangat besar dan biasnya kuat. Terlihat banyak kurtosis yang mendekati 0 dan tidak berdistribusi tajam, namun beberapa kurtosis memiliki data dengan sebaran tajam yang besar.

# Pendekatan untuk mengekstrak fitur penting (important Feature)

**Direction**

Untuk mencari fitur penting dari sejumlah besar variabel, analisis dilakukan dengan menggunakan tiga pendekatan berikut.
1) PCA dan analisis klaster
2) Pemeriksaan kepentingan oleh pengklasifikasi hutan acak

Dari hasil ini, kami memutuskan untuk membuat penilaian yang komprehensif dan memutuskan fitur-fitur penting.

Oleh karena itu, pertama-tama kami melakukan standarisasi data dan mengkonfirmasi distribusinya, terutama untuk yang memiliki rasio nol melebihi 15%. Melihat distribusi, kami mempertimbangkan variabel yang datanya memiliki varians dan kemungkinan ada jumlah sampel yang cukup.

Adapun nilai-nilai Null lainnya, nilai-nilai dengan kepentingan yang tersisa dalam analisis kepentingan variabel berikutnya diadopsi, dan diputuskan untuk mengisinya dengan nilai rata-rata yang memiliki pengaruh statistik paling kecil.

In [ ]:
# Data
label = train["class"]
X = train.iloc[:,1:]
col = train.iloc[:,1:].columns

# Scaling
# Create instance
sc = StandardScaler()
# Fitting
sc.fit(X)
# Transform
X_std = sc.fit_transform(X)

In [ ]:
## test data
X_test_std = sc.fit_transform(test.iloc[1:])
Y_test = test["class"]

In [ ]:
# Create data frame
train_std = pd.DataFrame(X_std, columns=col)
train_std["class"] = label
train_std.head()

In [ ]:
# Create test data frame
test_std = pd.DataFrame(X_test_std, columns=test.iloc[1:].columns)
test_std["class"] = Y_test
test_std.head()

In [ ]:
# Null value
null_df = pd.DataFrame({"variables":train.iloc[:,1:].columns,
                        "null_ratio":null_ratio})
null_over15_col = null_df[null_df["null_ratio"]>15]["variables"]

In [ ]:
# Result columns
null_over15_col

In [ ]:
# null ratio df
null_ratio_df = pd.DataFrame({"variables":train_std[null_over15_col].isnull().sum().index, 
                             "null_ratio":train_std[null_over15_col].isnull().sum()/len(train_std)*100}).sort_values(by="null_ratio", ascending=False)

# over 15% null value df
null_df = train_std[null_ratio_df["variables"]]

In [ ]:
# distribution check
col = null_df.columns
fig, ax = plt.subplots(4, 7, figsize=(25, 20))
plt.subplots_adjust(hspace=0.5)
for i in range(len(col)):
    if i <= 6:
        sns.distplot(null_df[col[i]], ax=ax[0,i], kde=False)
        ax[0,i].set_title("Null ratio(%):\n{}".format(null_ratio_df[null_ratio_df["variables"]==col[i]]["null_ratio"].values))
    if i > 6 and i <= 13:
        sns.distplot(null_df[col[i]], ax=ax[1,i-7], kde=False)
        ax[1,i-7].set_title("Null ratio(%):\n{}".format(null_ratio_df[null_ratio_df["variables"]==col[i]]["null_ratio"].values))
    if i > 13 and i <= 20:
        sns.distplot(null_df[col[i]], ax=ax[2,i-14], kde=False)
        ax[2,i-14].set_title("Null ratio(%):\n{}".format(null_ratio_df[null_ratio_df["variables"]==col[i]]["null_ratio"].values))
    if i > 20 and i <= 27:
        sns.distplot(null_df[col[i]], ax=ax[3,i-21], kde=False)
        ax[3,i-21].set_title("Null ratio(%):\n{}".format(null_ratio_df[null_ratio_df["variables"]==col[i]]["null_ratio"].values))

Sebagai hasil dari konfirmasi distribusi, diputuskan untuk meninggalkan bl_000 dan bk_000 karena rasio nilai nol adalah 40 hingga 50% dan ada lebih dari setengah data, dan data juga tersebar dan kemungkinan memiliki informasi. .. Variabel lain dikeluarkan karena jumlah datanya kecil dan biasnya hampir satu.

In [ ]:
# Columns of drop
drop_col = null_over15_col.values
drop_col = np.delete(drop_col, np.where((drop_col == 'bl_000') & (drop_col == 'bk_000')))
# Checking
drop_col

In [ ]:
# Drop over15% null columns, and fill mean values
train_std.drop(drop_col, axis=1, inplace=True)

# Roop fill mean
for i in train_std.columns:
    mean = train_std[i].mean()
    train_std[i].fillna(mean, inplace=True)
    
train_std.head()

In [ ]:
# Drop over15% null columns, and fill mean values
test_std.drop(drop_col, axis=1, inplace=True)

# Roop fill mean
for i in test_std.columns:
    mean = test_std[i].mean()
    test_std[i].fillna(mean, inplace=True)
    
test_std.head()

In [ ]:
train_std.corr().isnull().sum()

Beberapa data memiliki varians nol. Hilangkan karena tidak mempengaruhi hasil analisis dan menjadi risiko komputasi.

In [ ]:
# Sumber kesalahan yang terjadi selama analisis
train_std["cd_000"].var()

In [ ]:
train_std.drop("cd_000", axis=1, inplace=True)
# test data
test_std.drop("cd_000", axis=1, inplace=True)

In [ ]:
# Difine variables
X = train_std.iloc[:,:-1]
Y = train_std["class"]

# K-Means Clustering¶


Periksa dengan nomor cluster, dengan metode siku.

In [ ]:
# Calculation of distiortions
distortions = []
for i in range(1,20):
    km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=10)
    km.fit(X)
    distortions.append(km.inertia_)
    
# Plotting 
plt.figure(figsize=(10,6))
plt.plot(range(1,20), distortions, marker='o')
plt.xlabel("Number of clusters")
plt.xticks(range(1,20))
plt.ylabel("Distortion")

Dapat dilihat bahwa distorsi berkurang dengan bertambahnya jumlah cluster. Secara khusus, dapat dilihat bahwa jumlah cluster sangat berkurang menjadi 4 dan kemudian secara bertahap berkurang. Kali ini, kami memutuskan untuk melanjutkan analisis, dengan asumsi bahwa jumlah cluster di mana kurva ini asimtotik adalah hingga 8.

In [ ]:
# Clustering n=8
kmeans = KMeans(n_clusters=8, init='k-means++', n_init=10, max_iter=100, random_state=10)

In [ ]:
# Fitting
kmeans.fit(X)
# output
cluster = kmeans.labels_

In [ ]:
# test data fit transform and labels
cluster_test = kmeans.fit_predict(test_std.iloc[:,:-1])
print(cluster_test)

# PCA

Untuk menentukan jumlah komponen utama, kami mengkonfirmasi peluruhan nilai eigen dari varians dengan plot scree.

**Scree plot**


In [ ]:
eigen_vals = sorted(np.linalg.eigvals(X.corr()), reverse=True)

# plot
fig, ax = plt.subplots(2, 1, figsize=(20,10))
plt.subplots_adjust(hspace=0.4)
ax[0].plot(eigen_vals, 's-')
ax[0].set_xlabel("factor")
ax[0].set_ylabel("eigenvalue")

ax[1].plot(eigen_vals, 's-')
ax[1].set_xlabel("factor")
ax[1].set_ylabel("eigenvalue")
ax[1].set_ylim([0,10])
ax[1].set_title("Scale up")

Sebagai hasil dari konfirmasi nilai eigen menggunakan nilai varians, sangat mungkin bahwa bahkan dua dapat menjelaskan sebagian besar dari keseluruhan data, dan bahkan jika ada 10 variabel, seluruh data dapat diungkapkan dengan informasi yang cukup. Saya mengerti.

In [ ]:
# Create instance, n=10
pca = PCA(n_components=10)

# Fitting
pca_result = pca.fit_transform(X)
pca_result = pd.DataFrame(pca_result, columns=["pca1","pca2","pca3","pca4","pca5","pca6","pca7","pca8","pca9","pca10"])

pca_result.head()

In [ ]:
# Visualization by heatmap
plt.figure(figsize=(20,20))
sns.heatmap(pca.components_.T, vmax=1, vmin=-1, cmap='bwr', square=False, annot=False, center=0, yticklabels=X.columns, xticklabels=pca_result.columns)
plt.xlabel("pca")
plt.ylabel("variables")

Meskipun hubungan jumlah fitur berbeda untuk setiap komponen utama, kekuatan variabel yang terkait dengan informasi data dapat dilihat dari gambar ini. Dapat juga dilihat bahwa selama ini terdapat variabel-variabel yang tidak berhubungan dengan komponen utama.

In [ ]:
# Visualization by plot
x = pca_result["pca1"]
y = pca_result["pca2"]
color = Y

plt.figure(figsize=(10,6))
plt.scatter(x, y, c=color, alpha=0.5)
plt.xlabel("PCA1")
plt.ylabel("PCA2")
plt.colorbar()

Sebagian besar dari dua sumbu komponen utama tumpang tindih ketika diplot oleh label kelas, tetapi daerah dengan sejumlah besar kelas positif dan daerah dengan kelas negatif padat dan padat dipisahkan.

In [ ]:
# create dataframe
pca_result["class"] = Y

fig, ax = plt.subplots(1,2,figsize=(20,6))
sns.distplot(pca_result[pca_result["class"]==1]["pca1"], label="pos", ax=ax[0])
sns.distplot(pca_result[pca_result["class"]==0]["pca1"], label="neg", ax=ax[0])
ax[0].legend()

sns.distplot(pca_result[pca_result["class"]==1]["pca2"], label="pos", ax=ax[1])
sns.distplot(pca_result[pca_result["class"]==0]["pca2"], label="neg", ax=ax[1])
ax[1].legend()

Lebih mudah untuk memahami distribusi setiap sumbu komponen utama yang diplot. Mengenai sumbu pca1, kelas positif bias menuju nilai yang lebih besar, dan kelas negatif mengelompok di sekitar 0. Sumbu pca2 tersebar luas dari nilai negatif ke nilai positif di kelas positif, tetapi bias mendekati 0 di negatif kelas.

In [ ]:
pca_result["cluster"] = cluster

# Visualization by plot
x = pca_result["pca1"]
y = pca_result["pca2"]

plt.figure(figsize=(10,6))
plt.scatter(x, y, c=cluster, alpha=0.5, cmap="Set1")
plt.xlabel("PCA1")
plt.ylabel("PCA2")
plt.colorbar()

In [ ]:
# pivot count 
pivot = pd.pivot_table(data=pca_result, index="class", columns="cluster", values="pca1", aggfunc="count", fill_value=0)
pivot.columns = ["c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8"]
pivot.reset_index()

Selanjutnya, ketika hasil analisis klaster diplot, dipastikan bahwa klaster dapat dibagi bahkan pada sumbu komponen utama.
Apakah cluster ini membagi kelas positif dan negatif untuk label kelas? dikonfirmasi. Akibatnya, cluster 1 dan cluster 5 dapat membentuk cluster yang memiliki sejumlah besar kelas negatif. Ini adalah titik tajam di sisi kiri grafik plot. (Catatan: area kuning dan merah di plot)

Selanjutnya, saya juga mengkonfirmasi bagaimana variabel lain berbeda dalam cluster..
Namun, cluster 8 jumlahnya kecil dan merupakan outlier, sehingga dikecualikan.

In [ ]:
# Characteristics of each cluster
# Create dataframe
cluster_name = ["c1", "c2", "c3", "c4", "c5", "c6", "c7"]
pca_label = ["pca1","pca2","pca3","pca4","pca5","pca6","pca7","pca8","pca9","pca10"]

cluster_stats = pd.DataFrame({"cluster":range(0,7)})
cluster_pos_mean = pd.merge(cluster_stats, pca_result[pca_result["class"]==1].groupby("cluster").mean()[pca_label].reset_index(),
                            left_on="cluster", right_on="cluster", how="left").drop("cluster", axis=1)
cluster_pos_std = pd.merge(cluster_stats, pca_result[pca_result["class"]==1].groupby("cluster").std()[pca_label].reset_index(),
                            left_on="cluster", right_on="cluster", how="left").drop("cluster", axis=1)
cluster_neg_mean = pd.merge(cluster_stats, pca_result[pca_result["class"]==0].groupby("cluster").mean()[pca_label].reset_index(),
                            left_on="cluster", right_on="cluster", how="left").drop("cluster", axis=1)
cluster_neg_std = pd.merge(cluster_stats, pca_result[pca_result["class"]==0].groupby("cluster").std()[pca_label].reset_index(),
                            left_on="cluster", right_on="cluster", how="left").drop("cluster", axis=1)

# Change name of columns
cluster_pos_mean.index = cluster_name
cluster_pos_std.index = cluster_name
cluster_neg_mean.index = cluster_name
cluster_neg_std.index = cluster_name

In [ ]:
# Visualization
fig, ax = plt.subplots(2,5,figsize=(20,10))
plt.subplots_adjust(hspace=0.5, wspace=0.4)

for i in range(len(cluster_pos_mean.columns)):
    if i <5:
        ax[0,i].plot(cluster_pos_mean.index, cluster_pos_mean[pca_label[i]], color="red", label="pos")
        ax[0,i].fill_between(cluster_pos_mean.index, cluster_pos_mean[pca_label[i]]+cluster_pos_std[pca_label[i]],
                             cluster_pos_mean[pca_label[i]]-cluster_pos_std[pca_label[i]], color="orange", alpha=0.3, label="±1σ")
        ax[0,i].plot(cluster_neg_mean.index, cluster_neg_mean[pca_label[i]], color="blue", label="neg")
        ax[0,i].fill_between(cluster_neg_mean.index, cluster_neg_mean[pca_label[i]]+cluster_neg_std[pca_label[i]],
                             cluster_neg_mean[pca_label[i]]-cluster_neg_std[pca_label[i]], color="green", alpha=0.3, label="±1σ")
        ax[0,i].set_title(pca_label[i])
        ax[0,i].set_xlabel("Cluster")
        ax[0,i].set_ylabel("Standarlized values")
        ax[0,i].tick_params(axis='x', labelrotation=90)
        ax[0,i].legend(ncol=2)
    else:
        ax[1,i-5].plot(cluster_pos_mean.index, cluster_pos_mean[pca_label[i]], color="red", label="pos")
        ax[1,i-5].fill_between(cluster_pos_mean.index, cluster_pos_mean[pca_label[i]]+cluster_pos_std[pca_label[i]],
                               cluster_pos_mean[pca_label[i]]-cluster_pos_std[pca_label[i]], color="orange", alpha=0.3, label="±1σ")
        ax[1,i-5].plot(cluster_neg_mean.index, cluster_neg_mean[pca_label[i]], color="blue", label="neg")
        ax[1,i-5].fill_between(cluster_neg_mean.index, cluster_neg_mean[pca_label[i]]+cluster_neg_std[pca_label[i]],
                               cluster_neg_mean[pca_label[i]]-cluster_neg_std[pca_label[i]], color="green", alpha=0.3, label="±1σ")
        ax[1,i-5].set_title(pca_label[i])
        ax[1,i-5].set_xlabel("Cluster")
        ax[1,i-5].set_ylabel("Standarlized values")
        ax[1,i-5].tick_params(axis='x', labelrotation=90)
        ax[1,i-5].legend(ncol=2)

Sebagai hasil dari konfirmasi setiap komponen utama selain pca1 dan pca2, karakteristik komponen utama diklasifikasikan untuk setiap cluster. Point of interest adalah cluster 1 dan 5 yang sebagian besar termasuk dalam kelas negatif tadi, namun dengan melihat masing-masing sumbu dapat diketahui bahwa cluster lain dapat memiliki nilai yang sama. Artinya tidak mungkin membuat suatu penilaian dengan hanya satu variabel saja, hal ini menunjukkan perlunya membuat suatu penilaian positif/negatif yang mencakup hubungan antar masing-masing variabel.

Di sisi lain, cluster 4, 6 dan 7 adalah cluster yang memiliki sejumlah besar kelas positif, dan dapat dipastikan bahwa nilainya juga merupakan karakteristik. Faktanya, memahami karakteristik kluster ini akan membantu menganalisis mekanisme mekanis.

# Feature importance analysis dengan menggunakan Random forest

Selanjutnya, kami menganalisis pentingnya fitur menggunakan pendekatan yang sama sekali berbeda. Prediksi klasifikasi dilakukan di hutan acak, dan pentingnya fitur diekstraksi menggunakan perolehan informasi yang merupakan fitur dari pohon keputusan.

Ada masalah besar dalam mengklasifikasikan. Artinya, ada bias yang besar antara kelas positif dan negatif. Dalam kasus seperti itu, melengkapi menggunakan metode oversampling atau metode undersampling, dan menyeimbangkan setiap kelas untuk membuat model.
Kali ini, hanya metode oversampling yang digunakan karena hanya fitur kuantitas fitur yang penting. Juga, untuk referensi, sebagai hasil dari analisis klaster, klaster yang memiliki banyak kelas negatif dapat diapit. Oleh karena itu, kami mencoba membangun model dengan sampel data tidak termasuk cluster.

Untuk metode lain dari data yang tidak seimbang, efeknya diverifikasi ketika membangun model prediksi setelah analisis kepentingan ini.

**Konfirmasi pentingnya fitur, dengan metode oversampling**.

# Over sampling method

Eksekusi pengklasifikasi Random Forest

In [ ]:
# Difine variables
X = train_std.iloc[:,:-1]
Y = train_std["class"]

# Data preprocessing, oversampling method
# create instance
ros = RandomOverSampler(sampling_strategy="auto", random_state=10)

# train test data split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

# Apply to data
X_resampled, y_resampled = ros.fit_sample(X_train, y_train)


In [ ]:
# Create instance
forest  = RandomForestClassifier(n_estimators=10, random_state=10)

In [ ]:
# parameters
param_range = [10,15,20]
leaf = [70, 75, 80, 85]
criterion = ["entropy", "gini", "error"]
param_grid = [{"max_depth":param_range, "criterion":criterion, "max_leaf_nodes":leaf}]

# Optimization by Grid search, scoring is f1 score
gs = GridSearchCV(estimator=forest, param_grid=param_grid, scoring="f1", cv=5, n_jobs=-1)
gs = gs.fit(X_resampled, y_resampled)

In [ ]:
print("gs best:%.3f" % gs.best_score_)
print("gs params:{}".format(gs.best_params_))

In [ ]:
# Prediction
gs_best = gs.best_estimator_

y_pred = gs_best.predict(X_test)

In [ ]:
#Scores
print("Confusion_matrix = \n", confusion_matrix(y_true=y_test, y_pred=y_pred))
print("accuracy = %.3f" % accuracy_score(y_true=y_test, y_pred=y_pred))
print("precision = %.3f" % precision_score(y_true=y_test, y_pred=y_pred))
print("recall = %.3f" % recall_score(y_true=y_test, y_pred=y_pred))
print("f1_score = %.3f" % f1_score(y_true=y_test, y_pred=y_pred))

Hasil untuk data pelatihan memberikan ekstraksi yang solid dari kelas positif. Namun, skor presisi rendah karena seluruh data tidak seimbang. Meskipun ini merupakan masalah, dapat dievaluasi bahwa nilai recall tinggi dan kelas positif dapat diklasifikasikan untuk sementara waktu.
Dari hasil model ini diperoleh derajat kepentingan jumlah fitur.

In [ ]:
# Create best model of random forest classifier
forest  = RandomForestClassifier(n_estimators=10, random_state=10, criterion="entropy", max_depth=15, max_leaf_nodes=60)
forest.fit(X_resampled, y_resampled)

importance = forest.feature_importances_

indices = np.argsort(importance)[::-1]

# Due to the large number of items, only the top 30 were written.
for i in range(30):
    print("%2d) %-*s %f" %(i+1, 10, X.columns[indices[i]], importance[indices[i]]))

In [ ]:
# Visualization with paret0 graph
forest_importance1 = pd.DataFrame({})
variables = []
feature_importance1 = []
for i in range(len(indices)):
    col = X.columns[indices[i]]
    impor = importance[indices[i]]
    variables.append(col)
    feature_importance1.append(impor)
forest_importance1["variables"] = variables
forest_importance1["feature_importance1"] = feature_importance1
forest_importance1["feature_importance1"] = forest_importance1["feature_importance1"]*100
forest_importance1["cumsum"] = forest_importance1["feature_importance1"].cumsum()


# Graph
fig, ax1 = plt.subplots(figsize=(20,8))
ax1.bar(forest_importance1["variables"], forest_importance1["feature_importance1"], label="importance")
ax1.grid()
ax1.set_xlabel("variables")
ax1.tick_params(axis='x', rotation=90, labelsize=10)
ax1.set_ylabel("importance(%)")
plt.legend(loc='lower left')
ax2 = ax1.twinx()
ax2.plot(forest_importance1["variables"], forest_importance1["cumsum"], color="red", label="ratio")
ax2.set_ylim([0,110])
ax2.set_ylabel("Ratio(%)")
plt.legend(loc="upper right")

Tentang hasil klasifikasi hutan secara acak ini, ditemukan bahwa sekitar 1/4 variabel dapat menjelaskan lebih dari 90% klasifikasi, dan 2/3 variabel dapat menjelaskan hampir 100%.

# Analisis kepentingan fitur dengan menggunakan hasil Random Forest dan Clustering

Dari hasil analisis klaster diketahui bahwa kedua klaster tersebut hampir tidak mengandung label positif. Dengan menghapus cluster ini, jumlah kelas positif dan negatif didekatkan, dan analisis serupa dilakukan.

In [ ]:
train_std["cluster"] = cluster
# Create variable data and label data by removing the cluster data of c1 and c5 (0th and 4th in the label).

# test data
test_std["cluster"] = cluster_test

In [ ]:
# Difine variables
X = train_std.query("cluster!=0 & cluster!=4").iloc[:,:-2]
Y = train_std.query("cluster!=0 & cluster!=4")["class"]

# train test data split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

# Create instance
forest  = RandomForestClassifier(n_estimators=10, random_state=10)

# parameters
param_range = [5, 10,15]
leaf = [60, 65, 70, 75]
criterion = ["entropy", "gini", "error"]
param_grid = [{"max_depth":param_range, "criterion":criterion, "max_leaf_nodes":leaf}]

# Optimization by Grid search, scoring is f1
gs = GridSearchCV(estimator=forest, param_grid=param_grid, scoring="f1", cv=5, n_jobs=-1)
gs = gs.fit(X_train, y_train)

print("gs best:%.3f" % gs.best_score_)
print("gs params:{}".format(gs.best_params_))

In [ ]:
# Prediction
gs_best = gs.best_estimator_

y_pred = gs_best.predict(X_test)

# Scores
print("Confusion_matrix = \n", confusion_matrix(y_true=y_test, y_pred=y_pred))
print("accuracy = %.3f" % accuracy_score(y_true=y_test, y_pred=y_pred))
print("precision = %.3f" % precision_score(y_true=y_test, y_pred=y_pred))
print("recall = %.3f" % recall_score(y_true=y_test, y_pred=y_pred))
print("f1_score = %.3f" % f1_score(y_true=y_test, y_pred=y_pred))

Hasil untuk data pelatihan memberikan ekstraksi yang solid dari kelas positif. Skor presisi 84%. Meskipun ini merupakan masalah, dapat dievaluasi bahwa nilai recall rendah dan kelas positif dapat diklasifikasikan untuk sementara waktu. Dari hasil model ini diperoleh derajat kepentingan jumlah fitur.

In [ ]:
# Create best model of random forest classifier
forest  = RandomForestClassifier(n_estimators=10, random_state=10, criterion="entropy", max_depth=15, max_leaf_nodes=60)
forest.fit(X_train, y_train)

importance = forest.feature_importances_

indices = np.argsort(importance)[::-1]

# Due to the large number of items, only the top 30 were written.
for i in range(30):
    print("%2d) %-*s %f" %(i+1, 10, X.columns[indices[i]], importance[indices[i]]))

In [ ]:
# Visualization with paret0 graph
forest_importance2 = pd.DataFrame({})
variables = []
feature_importance2 = []
for i in range(len(indices)):
    col = X.columns[indices[i]]
    impor = importance[indices[i]]
    variables.append(col)
    feature_importance2.append(impor)
forest_importance2["variables"] = variables
forest_importance2["feature_importance2"] = feature_importance2
forest_importance2["feature_importance2"] = forest_importance2["feature_importance2"]*100
forest_importance2["cumsum"] = forest_importance2["feature_importance2"].cumsum()


# Graph
fig, ax1 = plt.subplots(figsize=(20,8))
ax1.bar(forest_importance2["variables"], forest_importance2["feature_importance2"], label="importance")
ax1.grid()
ax1.set_xlabel("variables")
ax1.tick_params(axis='x', rotation=90, labelsize=10)
ax1.set_ylabel("importance(%)")
plt.legend(loc='lower left')
ax2 = ax1.twinx()
ax2.plot(forest_importance2["variables"], forest_importance2["cumsum"], color="red", label="ratio")
ax2.set_ylim([0,110])
ax2.set_ylabel("Ratio(%)")
plt.legend(loc="upper right")

Dari hasil tersebut, kita dapat melihat peningkatan nilai presisi. Di sisi lain, nilai recall mengalami penurunan, namun secara keseluruhan skornya baik. Distribusi derajat kepentingan variabel telah berubah dari hasil sebelumnya. Garis dengan lebih dari 90% memiliki hampir setengah jumlah variabel, dan asimtotik hingga 100% sekitar 4/5. Dengan kata lain, variabel yang memberikan hasil yang tidak begitu penting dengan pelatihan secara keseluruhan menjadi lebih penting pada cluster dengan konsentrasi kelas positif yang tinggi, dan menjadi variabel yang diperlukan untuk klasifikasi. Itu mungkin untuk meningkatkan kinerja klasifikasi dengan menggabungkan analisis cluster.
Juga, jika ada variabel yang ditemukan tidak signifikan baik dalam hasil keseluruhan dan hasil ini, kemungkinan mereka dapat dikeluarkan dari model jika isinya dikonfirmasi dan penjelasan logis diberikan.

# Deteksi fitur yang tidak pnting (unimportant features)


In [ ]:
forest_importance = pd.merge(forest_importance1.drop("cumsum", axis=1), forest_importance2.drop("cumsum", axis=1),
                             left_on="variables", right_on="variables", how='left')
forest_importance


Kedua kepentingan menghitung jumlah masing-masing rasio.

In [ ]:
count = []
ratio = []
for i in range(0,11):
    r = i*0.1
    c = forest_importance[(forest_importance["feature_importance1"]<=i*0.1) & (forest_importance["feature_importance2"]<=i*0.1)]["variables"].count()
    count.append(c)
    ratio.append(r)
    
pd.DataFrame({"Both ratio(%)<":ratio,
              "count":count})

Mereka yang memiliki kedua kepentingan di bawah ambang batas masing-masing dihitung. Hasilnya, ada 13 variabel dengan kedua kepentingan sama-sama 0. Selain itu, 27 variabel sesuai dengan ambang 0,1% dan 42 variabel sesuai dengan ambang 0,2%.

Selanjutnya, hasil kinerja klasifikasi dengan model hutan acak ketika variabel dijatuhkan untuk setiap ambang batas dibandingkan.
Dari hasil keseluruhan data dan hasil cluster, variabel optimum dapat berubah tergantung kepentingannya, sehingga kondisi diset menjadi “atau”.

# Verifikasi dengan seluruh data

In [ ]:
# Variables
X = train_std.iloc[:,:-2]
Y = train_std["class"]

# Roop
Threshold = []
col_count = []
accuracy = []
precision = []
recall = []
f1_ = []

for i in range(0,10):
    # Create instance, parameters are default.
    forest  = RandomForestClassifier(n_estimators=10, random_state=10)
    
    # threshold
    thre = "> 0." + str(i) +"%"

    col = forest_importance[(forest_importance["feature_importance1"] >= i*0.1) | (forest_importance["feature_importance2"] >= i*0.1)]["variables"].values
    col_c = len(col)
    # Select data
    X = train_std[col]
    y = train_std["class"]
    # With over sampling
    # Data preprocessing, oversampling method
    # create instance
    ros = RandomOverSampler(sampling_strategy="auto", random_state=10)

    # train test data split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

    # Apply to data
    X_resampled, y_resampled = ros.fit_sample(X_train, y_train)
    
    # Fitting
    forest.fit(X_resampled, y_resampled)
    
    # Prediction
    y_pred = forest.predict(X_test)

    # Scores
    acc = accuracy_score(y_true=y_test, y_pred=y_pred).round(3)
    pre = precision_score(y_true=y_test, y_pred=y_pred).round(3)
    rec = recall_score(y_true=y_test, y_pred=y_pred).round(3)
    f1 = f1_score(y_true=y_test, y_pred=y_pred).round(3)
    
    # list append
    Threshold.append(thre)
    col_count.append(col_c)
    accuracy.append(acc)
    precision.append(pre)
    recall.append(rec)
    f1_.append(f1)

# create dataframe
pd.DataFrame({"Threshold":Threshold,
              "Col_count":col_count,
             "accuracy":accuracy,
             "precision":precision,
             "recall":recall,
             "f1_score":f1_})

# verifikasi seluruh data dan hasil clustering

In [ ]:
# Variables
X = train_std.iloc[:,:-2]
Y = train_std["class"]

# Roop
Threshold = []
col_count = []
accuracy = []
precision = []
recall = []
f1_ = []

for i in range(0,10):
    # Create instance, parameters are default.
    forest  = RandomForestClassifier(n_estimators=10, random_state=10)
    
    # threshold
    thre = "> 0." + str(i) +"%"

    col = forest_importance[(forest_importance["feature_importance1"] >= i*0.1) | (forest_importance["feature_importance2"] >= i*0.1)]["variables"].values
    col_c = len(col)
    # Select data
    X = train_std.query("cluster!=0 & cluster!=4")[col]
    y = train_std.query("cluster!=0 & cluster!=4")["class"]

    # train test data split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
    
    # Fitting
    forest.fit(X_train, y_train)
    
    # Prediction
    y_pred = forest.predict(X_test)

    # Scores
    acc = accuracy_score(y_true=y_test, y_pred=y_pred).round(3)
    pre = precision_score(y_true=y_test, y_pred=y_pred).round(3)
    rec = recall_score(y_true=y_test, y_pred=y_pred).round(3)
    f1 = f1_score(y_true=y_test, y_pred=y_pred).round(3)
    
    # list append
    Threshold.append(thre)
    col_count.append(col_c)
    accuracy.append(acc)
    precision.append(pre)
    recall.append(rec)
    f1_.append(f1)

# create dataframe
pd.DataFrame({"Threshold":Threshold,
              "Col_count":col_count,
             "accuracy":accuracy,
             "precision":precision,
             "recall":recall,
             "f1_score":f1_})

Melihat hasil menggunakan salah satu data pembelajaran, dapat dilihat bahwa pengurangan variabel tidak secara seragam mengurangi kinerja. Melihat kedua hasil tersebut, ditemukan bahwa hasil dengan threshold 0,6% adalah yang terbaik.

In [ ]:
best_col = forest_importance[(forest_importance["feature_importance1"] >= 0.6) | (forest_importance["feature_importance2"] >= 0.6)]["variables"].values
print(best_col)

# Analisis Fitur Penting menggunakan Random forest dan PCA variables

Selanjutnya, pentingnya sumbu komponen utama dianalisis dengan cara yang sama menggunakan hasil PCA di mana jumlah variabel dikurangi dengan pengurangan dimensi.

In [ ]:
# Difine variables
X = pca_result.iloc[:,:-2]
Y = pca_result["class"]

# train test data split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

# Data preprocessing, oversampling method
# create instance
ros = RandomOverSampler(sampling_strategy="auto", random_state=10)

# Apply to data
X_resampled, y_resampled = ros.fit_sample(X_train, y_train)

In [ ]:
# Create instance
forest  = RandomForestClassifier(n_estimators=10, random_state=10)

# parameters
param_range = [20, 25, 30]
leaf = [85, 90, 95, 100]
criterion = ["entropy", "gini", "error"]
param_grid = [{"max_depth":param_range, "criterion":criterion, "max_leaf_nodes":leaf}]

# Optimization by Grid search, scoring is f1
gs = GridSearchCV(estimator=forest, param_grid=param_grid, scoring="f1", cv=5, n_jobs=-1)
gs = gs.fit(X_resampled, y_resampled)

print("gs best:%.3f" % gs.best_score_)
print("gs params:{}".format(gs.best_params_))

In [ ]:
# Prediction
gs_best = gs.best_estimator_

y_pred = gs_best.predict(X_test)

# Scores
print("Confusion_matrix = \n", confusion_matrix(y_true=y_test, y_pred=y_pred))
print("accuracy = %.3f" % accuracy_score(y_true=y_test, y_pred=y_pred))
print("precision = %.3f" % precision_score(y_true=y_test, y_pred=y_pred))
print("recall = %.3f" % recall_score(y_true=y_test, y_pred=y_pred))
print("f1_score = %.3f" % f1_score(y_true=y_test, y_pred=y_pred))

In [ ]:
# Create best model of random forest classifier
forest  = RandomForestClassifier(n_estimators=10, random_state=10, criterion="entropy", max_depth=15, max_leaf_nodes=60)
forest.fit(X_resampled, y_resampled)

importance = forest.feature_importances_

indices = np.argsort(importance)[::-1]

# Karena banyaknya item, hanya 10 teratas yang ditulis.
for i in range(10):
    print("%2d) %-*s %f" %(i+1, 10, X.columns[indices[i]], importance[indices[i]]))

In [ ]:
# Visualization with paret0 graph
forest_importance = pd.DataFrame({})
variables = []
feature_importance = []
for i in range(len(indices)):
    col = X.columns[indices[i]]
    impor = importance[indices[i]]
    variables.append(col)
    feature_importance.append(impor)
forest_importance["variables"] = variables
forest_importance["feature_importance"] = feature_importance
forest_importance["feature_importance"] = forest_importance["feature_importance"]*100
forest_importance["cumsum"] = forest_importance["feature_importance"].cumsum()


# Graph
fig, ax1 = plt.subplots(figsize=(20,8))
ax1.bar(forest_importance["variables"], forest_importance["feature_importance"], label="importance")
ax1.grid()
ax1.set_xlabel("variables")
ax1.tick_params(axis='x', rotation=90, labelsize=10)
ax1.set_ylabel("importance(%)")
plt.legend(loc='lower left')
ax2 = ax1.twinx()
ax2.plot(forest_importance["variables"], forest_importance["cumsum"], color="red", label="ratio")
ax2.set_ylim([0,110])
ax2.set_ylabel("Ratio(%)")
plt.legend(loc="upper right")

Hasil analisis komponen utama menunjukkan tidak ada perbedaan kinerja klasifikasi yang signifikan dari hasil analisis menggunakan seluruh data. Tetapi yang penting adalah bahwa beberapa informasi dikurangi ketika memilih jumlah komponen utama. Sangat berguna untuk mendapatkan hasil yang serupa meskipun demikian, karena mudah untuk memahami keseluruhan gambar untuk analisis.

Meskipun hanya sekitar 1%, namun kecenderungan over-fitting pada data uji lebih kecil dibandingkan dengan data latih. Hal ini dapat dilihat karena jumlah variabel telah berkurang dan model menjadi lebih kuat. Metode berbasis PCA ini lebih cocok untuk operasi aktual dan prediksi data yang tidak diketahui.

#Membandingkan metode pengambilan sampel¶





Dari hasil analisis selama ini, ditemukan pentingnya membangun model dengan menggunakan seluruh data. Dalam hal ini, perlu diterapkan metode pengambilan sampel pada data yang tidak seimbang. Mulai sekarang, kami akan mengkonfirmasi efek dari beberapa metode ini dan mempertimbangkan yang terbaik. Prediksi menggunakan model hutan acak.

Metode yang dikonfirmasi adalah sebagai berikut:
Over sampling (Already confirmed)
Under sampling
SMOTE method
NearMiss method



**Under sampling**
Konfirmasikan hasil metode Under sampling di bawah kondisi yang sama seperti Oversampling, yang menangani semua data.

In [ ]:
# Difine variables
X = train_std.iloc[:,:-2]
Y = train_std["class"]

# Data preprocessing, oversampling method
# create instance
rus = RandomUnderSampler(sampling_strategy="auto", random_state=10)

# train test data split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

# Apply to data
X_resampled, y_resampled = rus.fit_sample(X_train, y_train)

In [ ]:
# Create instance
forest  = RandomForestClassifier(n_estimators=10, random_state=10)

# parameters
param_range = [5, 10,15,20]
leaf = [60, 65, 70, 75]
criterion = ["entropy", "gini", "error"]
param_grid = [{"max_depth":param_range, "criterion":criterion, "max_leaf_nodes":leaf}]

# Optimization by Grid search, scoring is f1 score
gs = GridSearchCV(estimator=forest, param_grid=param_grid, scoring="f1", cv=5, n_jobs=-1)
gs = gs.fit(X_resampled, y_resampled)

print("gs best:%.3f" % gs.best_score_)
print("gs params:{}".format(gs.best_params_))

In [ ]:
# Prediction
gs_best = gs.best_estimator_

y_pred = gs_best.predict(X_test)

# Scores
print("Confusion_matrix = \n", confusion_matrix(y_true=y_test, y_pred=y_pred))
print("accuracy = %.3f" % accuracy_score(y_true=y_test, y_pred=y_pred))
print("precision = %.3f" % precision_score(y_true=y_test, y_pred=y_pred))
print("recall = %.3f" % recall_score(y_true=y_test, y_pred=y_pred))
print("f1_score = %.3f" % f1_score(y_true=y_test, y_pred=y_pred))

**SMOTE**
mengonfirmasi metode SMOTE

In [ ]:
# Difine variables
X = train_std.iloc[:,:-2]
Y = train_std["class"]

# Data preprocessing, oversampling method
# create instance
smote = SMOTE(sampling_strategy="auto", random_state=10)

# train test data split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

# Apply to data
X_resampled, y_resampled = smote.fit_sample(X_train, y_train)

In [ ]:
# Create instance
forest  = RandomForestClassifier(n_estimators=10, random_state=10)

# parameters
param_range = [10,15,20]
leaf = [80, 85, 90, 95]
criterion = ["entropy", "gini", "error"]
param_grid = [{"max_depth":param_range, "criterion":criterion, "max_leaf_nodes":leaf}]

# Optimization by Grid search, scoring is f1 score
gs = GridSearchCV(estimator=forest, param_grid=param_grid, scoring="f1", cv=5, n_jobs=-1)
gs = gs.fit(X_resampled, y_resampled)

print("gs best:%.3f" % gs.best_score_)
print("gs params:{}".format(gs.best_params_))

In [ ]:
# Prediction
gs_best = gs.best_estimator_

y_pred = gs_best.predict(X_test)

# Scores
print("Confusion_matrix = \n", confusion_matrix(y_true=y_test, y_pred=y_pred))
print("accuracy = %.3f" % accuracy_score(y_true=y_test, y_pred=y_pred))
print("precision = %.3f" % precision_score(y_true=y_test, y_pred=y_pred))
print("recall = %.3f" % recall_score(y_true=y_test, y_pred=y_pred))
print("f1_score = %.3f" % f1_score(y_true=y_test, y_pred=y_pred))

**NearMiss method**

In [ ]:
# Difine variables
X = train_std.iloc[:,:-2]
Y = train_std["class"]

# Data preprocessing, oversampling method
# create instance
nem = NearMiss(sampling_strategy="auto")

# train test data split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

# Apply to data
X_resampled, y_resampled = nem.fit_sample(X_train, y_train)

In [ ]:
# Create instance
forest  = RandomForestClassifier(n_estimators=10, random_state=10)

# parameters
param_range = [5, 10,15,20]
leaf = [60, 65, 70, 75]
criterion = ["entropy", "gini", "error"]
param_grid = [{"max_depth":param_range, "criterion":criterion, "max_leaf_nodes":leaf}]

# Optimization by Grid search, scoring is f1 score
gs = GridSearchCV(estimator=forest, param_grid=param_grid, scoring="f1", cv=5, n_jobs=-1)
gs = gs.fit(X_resampled, y_resampled)

print("gs best:%.3f" % gs.best_score_)
print("gs params:{}".format(gs.best_params_))

In [ ]:
# Prediction
gs_best = gs.best_estimator_

y_pred = gs_best.predict(X_test)

# Scores
print("Confusion_matrix = \n", confusion_matrix(y_true=y_test, y_pred=y_pred))
print("accuracy = %.3f" % accuracy_score(y_true=y_test, y_pred=y_pred))
print("precision = %.3f" % precision_score(y_true=y_test, y_pred=y_pred))
print("recall = %.3f" % recall_score(y_true=y_test, y_pred=y_pred))
print("f1_score = %.3f" % f1_score(y_true=y_test, y_pred=y_pred))

Kami mengkonfirmasi metode pengambilan sampel untuk data yang tidak seimbang dalam kondisi yang sama. Jika dibandingkan dengan skor f1 untuk melihat keseimbangan antara presisi dan recall, metode over sampling memberikan hasil terbaik untuk data ini. Di sisi lain, undersampling dan turunannya, metode Near Miss, memiliki nilai recall yang tinggi tetapi nilai presisi yang rendah. Ini mungkin karena kekuatan negatif dari negatif dikurangi dengan mengurangi jumlah kelas negatif dan harus dihindari untuk data ini.

# Model Pengklasifikasi berdasarkan hasil sejauh ini

Berdasarkan hasil analisis sampai saat ini, kami memutuskan untuk mengambil kebijakan berikut mengenai penanganan data variabel dalam klasifikasi.

(1) Kurangi jumlah variabel yang tidak penting, dengan nilai threshold fitur penting 0,6%. Saya dapat mengharapkan efek pada PCA juga, tetapi kali ini saya memutuskan untuk melanjutkan dengan pengurangan variabel berdasarkan kepentingan. (Mungkin kita bisa mengharapkan peningkatan lebih lanjut dengan menggabungkannya ...)
(2) Analisis klaster dilakukan pada variabel yang direduksi untuk membuat kelompok klaster yang kelas positif dan negatifnya relatif seimbang.
(3) Latih model prediktif pada seluruh data dan data klaster yang disempurnakan dan gabungkan hasilnya. Untuk model yang menangani seluruh data, metode pengambilan sampel untuk data tidak seimbang adalah over sampling yang digunakan.

# Classification prediction model¶

**Prediction model : I use LGBM classifier**
**Difine class and function, k-fold cross validation, learning and validation curve, confusion matrix, roc auc curve**


In [ ]:
class k_fold_cross_val:
    def __init__(self, X_train, y_train, estimator, cv):
        self.X_train = X_train
        self.y_train = y_train
        self.estimator = estimator
        self.cv = cv
        
    def cross_val_kfold(self):
        kfold = StratifiedKFold(n_splits=self.cv, random_state=10)
        self.kfold = kfold
        
        scores = []
        for train_idx, test_idx in self.kfold.split(self.X_train, self.y_train):
            self.estimator.fit(self.X_train[train_idx], self.y_train.values[train_idx])
            score = self.estimator.score(self.X_train[test_idx], self.y_train.values[test_idx])
            scores.append(score)
            print("Class: %s, Acc: %.3f" % (np.bincount(self.y_train.values[train_idx]), score))
            self.scores = scores
            
    def score(self):
        scores = cross_val_score(estimator=self.estimator, X=self.X_train, y=self.y_train, cv=self.cv, n_jobs=1)
        print("CV accuracy scores: %s" % self.scores)
        print("CV accuracy: %.3f +/- %.3f" % (np.mean(self.scores), np.std(self.scores)))
        
    def draw_roc_curve(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
        
        mean_tpr=0
        mean_fpr=np.linspace(0,1,100)
        plt.figure(figsize=(10,6))
        for train_idx, test_idx in self.kfold.split(self.X_train, self.y_train):
            proba = self.estimator.fit(self.X_train[train_idx], self.y_train.values[train_idx]).predict_proba(self.X_train[test_idx])
            fpr, tpr, thresholds = roc_curve(y_true=self.y_train.values[test_idx], y_score=proba[:,1], pos_label=1)
            mean_tpr += interp(mean_fpr, fpr, tpr)
            mean_tpr[0] = 0
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, lw=1, label="ROC fold (area=%.2f)" %(roc_auc))
        
        # Line
        plt.plot([0,1], [0,1], linestyle='--', color=(0.6,0.6,0.6), label="random guessing")
        # plot mean of fpr, tpr roc_auc
        mean_tpr /= self.cv
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        plt.plot(mean_fpr, mean_tpr, 'k--', label="mean ROC (area = %.2f)" % mean_auc, color="blue")
        # Line
        plt.plot([0,0,1], [0,1,1], lw=2, linestyle=':', color="black", label='perfect performance')
        plt.xlabel("false positive rate")
        plt.ylabel("true positive rate")
        plt.title("Receiver Operator Characteristic")
        plt.legend(loc="upper right")
        

In [ ]:
def draw_learning_curve(estimator, X_train, y_train):
    # learning curve
    train_sizes, train_scores, test_scores = learning_curve(estimator=estimator, X=X_train, y=y_train, train_sizes=np.linspace(0.1,1,10), cv=10, n_jobs=1)
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    # plot
    plt.figure(figsize=(10,6))
    # train data
    plt.plot(train_sizes, train_mean, color="blue", marker='o', markersize=5, label='training accuracy')
    plt.fill_between(train_sizes, train_mean+train_std, train_mean-train_std, color="blue", alpha=0.15)
    # val data
    plt.plot(train_sizes, test_mean, color="green", marker='s', linestyle='--', markersize=5, label='validation accuracy')
    plt.fill_between(train_sizes, test_mean+test_std, test_mean-test_std, color="green", alpha=0.15)

    plt.grid()
    plt.xlabel("Number of trainig samples")
    plt.ylabel("Accuracy")
    plt.ylim([0.8,1.0])
    plt.title("Learning curve")
    plt.legend(loc="upper right")

In [ ]:
def draw_validation_curve(estimator, X_train, y_train, param_name, param_range, xscale):
    # validation curve
    train_scores, test_scores = validation_curve(estimator=estimator, X=X_train, y=y_train, param_name=param_name, param_range=param_range, cv=10)
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    
    # plot
    plt.figure(figsize=(10,6))
    # train data
    plt.plot(param_range, train_mean, color="blue", marker='o', markersize=5, label='training accuracy')
    plt.fill_between(param_range, train_mean+train_std, train_mean-train_std, color="blue", alpha=0.15)
    # val data
    plt.plot(param_range, test_mean, color="green", marker='s', linestyle='--', markersize=5, label='validation accuracy')
    plt.fill_between(param_range, test_mean+test_std, test_mean-test_std, color="green", alpha=0.15)

    plt.grid()
    plt.xlabel("{}".format(param_name))
    if xscale=="log":
        plt.xscale("log")
    else:
        pass
    plt.ylabel("Accuracy")
    plt.ylim([0.8,1.0])
    plt.title("Validation curve")
    plt.legend(loc="upper right")

**Parameters**

In [ ]:
best_col

#1st Prediksi Seluruh Data


In [ ]:
# Selected parameters, feature importance threshold >=0.6
col = best_col
# Variables
X = train_std[col]
y = train_std["class"]

# train test data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Apply over sampling method
ros = RandomOverSampler(sampling_strategy="auto", random_state=10)
X_resampled, y_resampled = ros.fit_sample(X_train, y_train)

# Study best parameter by Cross validation
# Instance
lgb_ = lgb.LGBMClassifier()

# prameters
max_depth = [5, 10, 15]
min_samples_leaf = [1,3,5,7]
min_samples_split = [4,6, 8, 10]

param_grid = [{"max_depth":max_depth,
               "min_samples_leaf":min_samples_leaf, "min_samples_split":min_samples_split}]

# Optimization by Grid search
gs = GridSearchCV(estimator=lgb_, param_grid=param_grid, scoring="f1", cv=5, n_jobs=-1)
gs = gs.fit(X_resampled, y_resampled)

print("gs best:%.3f" % gs.best_score_)
print("gs params:{}".format(gs.best_params_))

In [ ]:
 # best params
gs_best_all = gs.best_estimator_

# Cross validation
cv = k_fold_cross_val(X_resampled.values, y_resampled, gs_best_all, 5)
cv.cross_val_kfold()

In [ ]:
# cross val score
cv.score()

In [ ]:
# learning curve
draw_learning_curve(gs_best_all, X_resampled, y_resampled)

In [ ]:
# validation curve
draw_validation_curve(gs_best_all, X_resampled, y_resampled, "max_depth", param_range, ""), X_resampled, y_resampled, "max_depth", param_range, "")

In [ ]:
# cv training roc curve
cv.draw_roc_curve(X_resampled, y_resampled)


In [ ]:
# test data prediction
y_pred_all = gs_best_all.predict(X_test)

# Confusion matrix and ROC curve
confmat_roccurve(X_test, y_test, y_pred_all, gs_best_all)

# 2. Dengan prediksi data analisis cluster

In [ ]:
# Selected parameters, feature importance threshold >=0.6
col = best_col
# Variables
X = train_std.query("cluster!=0 & cluster!=4")[col]
y = train_std.query("cluster!=0 & cluster!=4")["class"]

# train test data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Apply over sampling method
ros = RandomOverSampler(sampling_strategy="auto", random_state=10)
X_resampled, y_resampled = ros.fit_sample(X_train, y_train)

# Study best parameter by Cross validation
# Instance
lgb_ = lgb.LGBMClassifier()

# prameters
max_depth = [5, 10, 15]
min_samples_leaf = [1,3,5,7]
min_samples_split = [4,6, 8, 10]

param_grid = [{"max_depth":max_depth,
               "min_samples_leaf":min_samples_leaf, "min_samples_split":min_samples_split}]

# Optimization by Grid search
gs = GridSearchCV(estimator=lgb_, param_grid=param_grid, scoring="f1", cv=5, n_jobs=-1)
gs = gs.fit(X_resampled, y_resampled)

print("gs best:%.3f" % gs.best_score_)
print("gs params:{}".format(gs.best_params_))

In [ ]:
# best params
gs_best_cluster = gs.best_estimator_

# Cross validation
cv = k_fold_cross_val(X_resampled.values, y_resampled, gs_best_cluster, 5)
cv.cross_val_kfold()

In [ ]:
# cross val score
cv.score()

In [ ]:
# learning curve
draw_learning_curve(gs_best_cluster, X_resampled, y_resampled)

In [ ]:
# validation curve
draw_validation_curve(gs_best_cluster, X_resampled, y_resampled, "max_depth", param_range, "")

In [ ]:
# cv training roc curve
cv.draw_roc_curve(X_resampled, y_resampled)

In [ ]:
# test data prediction
y_pred_cluster = gs_best_cluster.predict(X_test)

# Confusion matrix and ROC curve
confmat_roccurve(X_test, y_test, y_pred_cluster, gs_best_cluster)

# Pengujian prediksi data Buat prediksi ensemble¶

**Preprocessing** 
Seluruh data

In [ ]:
# Selected parameters, feature importance threshold >=0.6
col = best_col
# Variables
X_Test = test_std[col]
y_Test = test_std["class"]

# test data prediction
y_Test_pred_all = gs_best_all.predict(X_Test)

**custer data**

In [ ]:
# Variables
X_Test_cluster = test_std.query("cluster!=0 & cluster!=4")[col]
y_Test_cluster = test_std.query("cluster!=0 & cluster!=4")["class"]

# test data prediction
y_Test_pred_cluster = gs_best_cluster.predict(X_Test_cluster)

# Perbandingan setiap hasil prediksi

In [ ]:
# Scores
# All data prediction
print("-"*30, "all data", "-"*30)
print("Confusion_matrix = \n", confusion_matrix(y_true=y_Test, y_pred=y_Test_pred_all))
print("accuracy = %.3f" % accuracy_score(y_true=y_Test, y_pred=y_Test_pred_all))
print("precision = %.3f" % precision_score(y_true=y_Test, y_pred=y_Test_pred_all))
print("recall = %.3f" % recall_score(y_true=y_Test, y_pred=y_Test_pred_all))
print("f1_score = %.3f" % f1_score(y_true=y_Test, y_pred=y_Test_pred_all))

# with cluster data prediction
print("-"*30, "cluster data", "-"*30)
print("Confusion_matrix = \n", confusion_matrix(y_true=y_Test_cluster, y_pred=y_Test_pred_cluster))
print("accuracy = %.3f" % accuracy_score(y_true=y_Test_cluster, y_pred=y_Test_pred_cluster))
print("precision = %.3f" % precision_score(y_true=y_Test_cluster, y_pred=y_Test_pred_cluster))
print("recall = %.3f" % recall_score(y_true=y_Test_cluster, y_pred=y_Test_pred_cluster))
print("f1_score = %.3f" % f1_score(y_true=y_Test_cluster, y_pred=y_Test_pred_cluster))

# Kesimpulan

Hasilnya sesuai dengan kedua baris ke data pelatihan, tetapi tidak berhasil mendeteksi positif ke data uji. Hasilnya adalah ada kecenderungan yang kuat untuk belajar berlebihan.
Hipotesis saya dan tindakan balasan sebagai penyebabnya adalah

Bukankah variabel data uji tidak terkandung dalam data pelatihan di ruang dimensi tinggi dan ada banyak elemen di luar? Oleh karena itu, ada kemungkinan bahwa bahkan variabel penting berkurang ketika variabel dikurangi. Penanggulangan: Turunkan ambang batas pengurangan variabel.
Masih banyak variabel, dan beberapa variabel dalam data latih yang memiliki pengaruh kuat terlalu kuat dalam data uji. Penanggulangan: Menyamakan pengaruh variabel dengan analisis komponen utama.
Untuk uji coba selanjutnya, saya akan mencoba dua di atas.